In [1]:
# The goal is to predict price

# https://www.youtube.com/watch?v=HdlDYng8g9s&t=458s

import numpy as np
import pandas as pd
import sklearn.model_selection
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
sns.set_context('talk')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report
from sklearn import metrics 
from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

# https://machinelearningmastery.com/repeated-k-fold-cross-validation-with-python/
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv('C:\\Users\\Trey\\Desktop\\Capstone II\\Data\\winedata.csv')
df.head(5)

,country,description,designation,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,35.0,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,unknown,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",unknown,87,14.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,unknown,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


# Logistic Regression

In [3]:
# Going to work with less data for sake of time
df = df.head(5000)

In [4]:
print(df.price.describe())
print('10% '+ str(np.quantile(df.price, .1)))
print('90% '+ str(np.quantile(df.price, .9)))

count    5000.000000
mean       34.751800
std        50.495459
min         4.000000
25%        17.000000
50%        26.000000
75%        40.000000
max      1900.000000
Name: price, dtype: float64
10% 12.0
90% 60.0


In [5]:
df['priceCategory'] = df['price']

In [6]:
# change the variables to categories
for i in range(0, len(df.priceCategory)):
    if 13 > df.priceCategory[i] >= 4:
        df.priceCategory[i] = 'cheap'
    elif 18 > df.priceCategory[i] >= 13:
        df.priceCategory[i] = 'below average'
    elif 40 > df.priceCategory[i] >= 18:
        df.priceCategory[i] = 'average'
    elif 60 > df.priceCategory[i] >= 40:
        df.priceCategory[i] = 'above average'
    else:
        df.priceCategory[i] = 'expensive'

<ipython-input-6-0dc0336b92d7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.priceCategory[i] = 'average'
C:\Users\Trey\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-6-0dc0336b92d7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.priceCategory[i] = 'below average'
<ipython-input-6-0dc0336b92d7>:12: SettingWithCop

In [7]:
df.priceCategory.value_counts()

average          2439
below average     753
above average     714
cheap             554
expensive         540
Name: priceCategory, dtype: int64

In [8]:
# https://www.datasklr.com/logistic-regression/multinomial-logistic-regression
X = df[['points']] 
y = df['priceCategory']

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.20, random_state = 5)

model1 = LogisticRegression(random_state=0, multi_class='multinomial', penalty='none', solver='newton-cg').fit(X_train, y_train)
preds = model1.predict(X_test)

print('Accuracy Score:', metrics.accuracy_score(y_test, preds))  

Accuracy Score: 0.513


In [9]:
# now with categorical variables
# https://datascienceplus.com/building-a-logistic-regression-in-python-step-by-step/

ohe = OneHotEncoder()
column_trans = make_column_transformer((ohe,
    ['country', 'designation', 'province', 'region_1', 'winery', 'variety', 'taster_name']),
    remainder='passthrough')

X = df.drop(columns=['description', 'price', 'priceCategory', 'title', 'taster_twitter_handle'])
X.dtypes

country        object
designation    object
points          int64
province       object
region_1       object
taster_name    object
variety        object
winery         object
dtype: object

In [10]:
cols = ['country', 'designation', 'province', 'region_1', 'winery', 'variety', 'taster_name']
X[cols] = X[cols].astype('category')
X.dtypes

country        category
designation    category
points            int64
province       category
region_1       category
taster_name    category
variety        category
winery         category
dtype: object

In [11]:
X = column_trans.fit_transform(X)
y = df['priceCategory']

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.20, random_state = 5)

model1 = LogisticRegression(random_state=0, multi_class='multinomial', penalty='none', solver='newton-cg').fit(X_train, y_train)
preds = model1.predict(X_test)

print('Accuracy Score:', metrics.accuracy_score(y_test, preds))  

Accuracy Score: 0.489


### Tuning

In [12]:
# https://www.youtube.com/watch?v=pooXM9mM7FU
param_grid = [
    {'C' : [0.1, 1, 10, 100],
    'solver' : ['lbfgs', 'newton-cg'],
    'max_iter' : [1000, 4000]}
]
lrm = LogisticRegression(multi_class='multinomial')

In [13]:
clf = GridSearchCV(lrm, param_grid = param_grid, cv=5, n_jobs = -1)
clf_fitted = clf.fit(X,y)

In [14]:
clf_fitted.best_estimator_

LogisticRegression(C=1, max_iter=1000, multi_class='multinomial',
                   solver='newton-cg')

In [15]:
clf_fitted.score(X,y)

0.9192

# Random Forest

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score

In [17]:
# https://www.youtube.com/watch?v=aXw12xFVxwg

rf = RandomForestClassifier(n_estimators=100, criterion='entropy')
rf.fit(X_train, y_train)
prediction_test = rf.predict(X=X_test)

# Accuracy on Test
print("Training Accuracy is: ", rf.score(X_train, y_train))
# Accuracy on Train
print("Testing Accuracy is: ", rf.score(X_test, y_test))

Training Accuracy is:  0.99975
Testing Accuracy is:  0.593


### Tuning

In [18]:
param_grid = [
    {'n_estimators' : [50, 100, 200],
    'criterion' : ['gini', 'entropy'],
    'max_features' : [1, 'sqrt', 'log2'],
    'max_depth' : [None, 2, 5]}
]
rf = RandomForestClassifier()

In [19]:
rf_grid = GridSearchCV(rf, param_grid = param_grid, cv=5)
rf_grid_fitted = rf_grid.fit(X,y)

In [20]:
rf_grid_fitted.best_estimator_

RandomForestClassifier(max_features='sqrt', n_estimators=200)

In [21]:
rf_grid_fitted.score(X,y)

0.9992

# XGBoost

### Tuning

In [22]:
import xgboost

In [ ]:
classifier=xgboost.XGBClassifier(tree_method='gpu_hist')

params={
    "learning_rate":[0.05,0.15,0.30],
    "max_depth":[2,3,10,15],
    "min_child_weight":[1,3,7],
    "gamma":[0.0,0.1,0.3],
    "colsample_bytree":[0.30,0.5,0.7]}

xgclf = GridSearchCV(classifier,param_grid=params,cv=5,verbose=3)

xgclf_fitted = xgclf.fit(X,y)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
[CV] colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=2, min_child_weight=1 
[17:38:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\Trey\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV]  colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=2, min_child_weight=1, score=0.530, total=   6.2s
[CV] colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=2, min_child_weight=1 
[17:38:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.1s remaining:    0.0s


[CV]  colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=2, min_child_weight=1, score=0.525, total=   6.1s
[CV] colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=2, min_child_weight=1 
[17:38:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   12.2s remaining:    0.0s


[CV]  colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=2, min_child_weight=1, score=0.534, total=   6.1s
[CV] colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=2, min_child_weight=1 
[17:38:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV]  colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=2, min_child_weight=1, score=0.528, total=   6.1s
[CV] colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=2, min_child_weight=1 
[17:38:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric i

[CV]  colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, score=0.545, total=  12.7s
[CV] colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1 
[17:40:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV]  colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, score=0.532, total=  12.6s
[CV] colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=3 
[17:41:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric i

In [ ]:
xgclf_fitted.best_estimator_

In [ ]:
xgclf_fitted.score(X,y)